# 处理Movielens数据集

注意点：

由于电影id最大编号过大，为节省内存空间，使用movies中movieId相对应的index构建rating
遍历函数 iterrows()的使用:**iterrows()是在数据框中的行进行迭代的一个生成器，它返回每行的索引及一个包含行本身的对象**

In [3]:
import numpy as np
import pandas as pd

ratings_df = pd.read_csv('ratings.txt')
movies_df = pd.read_csv('movies.txt')

movies_df['movieRow'] = movies_df.index
movies_df = movies_df[['movieRow', 'movieId', 'title']]

ratings_df = pd.merge(ratings_df, movies_df,on='movieId')
ratings_df = ratings_df[['userId', 'movieRow','rating']]

userNo = ratings_df['userId'].max()+1
movieNo = ratings_df['movieRow'].max()+1

rating = np.zeros((movieNo, userNo))
rating_df_length = np.shape(ratings_df)[0]

flag = 0
for index,row in ratings_df.iterrows():
    rating[int(row['movieRow']),int(row['userId'])] = row['rating']
    flag += 1
    # print('processed %d, %d left' % (flag, rating_df_length - flag))
print(flag)

100836


# 内置数据集

# 1 自动交叉验证

Surprise有很多内置算法和数据集供你使用。最简单的，我们可以只用几行代码运行交叉验证程序。

In [4]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

date = Dataset.load_builtin('ml-100k')

algo = SVD()

cross_validate(algo,date,measures=['RMSE', 'MAE'],cv=5,verbose=True)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /Users/zhaoyadong/.surprise_data/ml-100k
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9354  0.9353  0.9442  0.9375  0.9294  0.9363  0.0048  
MAE (testset)     0.7362  0.7370  0.7443  0.7373  0.7330  0.7376  0.0037  
Fit time          4.72    4.81    4.31    4.64    4.05    4.51    0.28    
Test time         0.14    0.11    0.11    0.13    0.12    0.12    0.01    


{'test_rmse': array([0.9353563 , 0.93534871, 0.94416866, 0.93745895, 0.92935282]),
 'test_mae': array([0.73617369, 0.73704907, 0.74429304, 0.73728682, 0.73300928]),
 'fit_time': (4.72294282913208,
  4.814671039581299,
  4.314984083175659,
  4.637763977050781,
  4.051350831985474),
 'test_time': (0.13827300071716309,
  0.11265063285827637,
  0.11022210121154785,
  0.12871813774108887,
  0.11823582649230957)}

load_builtin()将下载movielens-100k数据集（尚未下载），并将其保存.surprise_data在主目录中的文件夹中（也可以将其保存在其他地方）。

我们使用著名的 SVD 算法，但还有许多其他算法可用。参考https://surprise.readthedocs.io/en/stable/prediction_algorithms.html#prediction-algorithms

cross_validate() 函数根据cv参数运行交叉验证程序，并计算一些accuracy进行评估。我们使用经典的5倍交叉验证，也可以使用优秀的迭代器https://surprise.readthedocs.io/en/stable/model_selection.html#cross-validation-iterators-api

# 2 Train-test split 和 fit() 方法

你如果不想使用交叉验证方法，也可以用 train_test_split() 来得到指定大小的测试集和训练集，并且可自己选择测量精度的方法 

accuracy metric 。训练时使用 fit() 在训练集训练，测试时用 test()，它将会返回测试集上的预测结果。

In [5]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import accuracy

data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(date,test_size=.25) #随机选取25%作为测试集
algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.9378


0.9377580946889902

In [6]:
# 你也可以用一行代码来进行测试和训练：
# predictions = algo.fit(trainset).test(testset)

有时候，训练集和测试集是给定的，需要这样处理：
https://surprise.readthedocs.io/en/stable/getting_started.html#load-from-folds-example

# 3 在整个训练集上训练和predict () 方法

显然，我们可以简单地将我们的算法作用于整个数据集，而不进行交叉验证。

这可以通过使用build_full_trainset() 建立 trainset 对象来完成 ：

In [7]:
from surprise import KNNBasic
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import accuracy

data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
algo = KNNBasic()
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


那么现在就可以通过 predict() 来预测评分。假设你对用户id=196和电影id=302（确定它们在训练集中）感兴趣，并且你也知道实际评分r_ui=4：

In [9]:
uid = str(196)
iid = str(302)

pred = algo.predict(uid,iid,r_ui=4,verbose=True)

user: 196        item: 302        r_ui = 4.00   est = 4.06   {'actual_k': 40, 'was_impossible': False}


注意：predict() 使用的是原始id（阅读这里了解原始id与内部id）由于数据集是由文件中读取出来的，因此原始id是字符串形式。

目前为止我们一直使用的是内置的数据集，当然你可以使用自己的数据集。下一部分我们会讲到这个用法。

# 使用自定义数据集

Surprise有一些内置数据集，但您当然可以使用自定义数据集。评分数据可以从文件（如csv文件）或pandas 的Dataframe 来加载。无论哪种方法，都需要定义一个Reader对象来解析文件或Dataframe。

* 1. 从一个文件中上传数据集（例如csv文件），你需要 load_from_file()函数。

In [10]:
from surprise import BaselineOnly
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import Reader
import os

file_path = os.path.expanduser('./ml-100k/u.data')
reader = Reader(line_format='user item rating timestamp', sep='\t')
data = Dataset.load_from_file(file_path, reader=reader)
cross_validate(BaselineOnly(),data,verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9566  0.9434  0.9372  0.9375  0.9458  0.9441  0.0071  
MAE (testset)     0.7574  0.7490  0.7408  0.7448  0.7503  0.7485  0.0056  
Fit time          0.22    0.18    0.18    0.18    0.18    0.19    0.02    
Test time         0.10    0.09    0.18    0.09    0.17    0.12    0.04    


{'test_rmse': array([0.95660034, 0.94335678, 0.93723187, 0.93752905, 0.94575436]),
 'test_mae': array([0.75737066, 0.74902301, 0.74077921, 0.74480377, 0.75027981]),
 'fit_time': (0.21695685386657715,
  0.17502593994140625,
  0.18213391304016113,
  0.17672276496887207,
  0.18005609512329102),
 'test_time': (0.09678196907043457,
  0.08768415451049805,
  0.18029189109802246,
  0.08571577072143555,
  0.1709728240966797)}

In [11]:
data

Reader更多的用法：
https://surprise.readthedocs.io/en/stable/reader.html#surprise.reader.Reader

* 2. 从pandas 的Dataframe 中获取数据，你需要load_from_df()函数。

你还需要一个Reader对象，只需要设定参数rating_scale参数即可。**dataframe必须包括三列，分别按顺序对应用户id，物品id以及评分。**

In [13]:
import pandas as pd
from surprise import NormalPredictor
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import Reader


ratings_dic = {'itemid':[1, 1, 1, 2, 2],
               'userid':[9, 32, 2, 45, 'user_foo'],
               'rating':[2, 2, 4, 3, 1]}

df = pd.DataFrame(ratings_dic)
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(df[['userid','itemid','rating']], reader)
cross_validate(NormalPredictor(),data,cv=2,verbose=True)

Evaluating RMSE, MAE of algorithm NormalPredictor on 2 split(s).

                  Fold 1  Fold 2  Mean    Std     
RMSE (testset)    1.4142  2.2300  1.8221  0.4079  
MAE (testset)     1.3333  1.9863  1.6598  0.3265  
Fit time          0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    


{'test_rmse': array([1.41421356, 2.22999725]),
 'test_mae': array([1.33333333, 1.98625495]),
 'fit_time': (8.177757263183594e-05, 5.1021575927734375e-05),
 'test_time': (5.412101745605469e-05, 2.8133392333984375e-05)}

# 使用交叉验证迭代器

可以使用cross_validate()函数做所有对我们来说困难的工作。但是为了更好的控制，我们实例化一个交叉验证迭代器，并使用迭代器的split()函数和test()函数在每一个split上做出预测。

如下，这是一个将数据集分为3折的经典K-折交叉验证实例。

In [14]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import KFold
from surprise import accuracy


data = Dataset.load_builtin('ml-100k')
kf = KFold(n_splits=3)
algo = SVD()

for trainset,testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.9468


0.9468078620679555

RMSE: 0.9423


0.9422800130171692

RMSE: 0.9445


0.944466318571693

其他验证器，例如**留一法**等交叉验证器也可使用，可以在这里查看所有的交叉验证器。

我们有时会遇到数据集已经被预划分为几个文件，例如 movielens-100k就分别提供了5个训练集和测试集。

Surprise可以使用surprise.model_selection.split.PredefinedKFold对象来处理这种问题。（情况较少）

# 使用GridSearchCV调整算法参数

cross_validate()函数可以对一组给定参数的交叉验程序进行准确性测评。如果你想知道哪些参数组合能够产生最好的结果，那么这个 GridSearchCV类就可以帮助你。给定一个参数dic，它会测试所有的参数组合，并报告对所有评测指标的最佳参数。这个想法受到scikit-learn的GridSearchCV的启发。

下面这个实例，我们测试了SVD算法的参数 `n_epochs`, `lr_all` 和 `reg_all`的不同值。

In [15]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import GridSearchCV


data = Dataset.load_builtin('ml-100k')
para_grid = {'n_epochs':[5,10],
             'lr_all':[0.002,0.005],
             'reg_all':[0.4,0.6]}
gs = GridSearchCV(SVD, para_grid, measures=['rmse','mae'],cv=3)
gs.fit(data)

#获得rmse最好得分
print(gs.best_score['rmse'])

#获得使rmse最好得分的参数
print(gs.best_params['rmse'])

0.9645338228067936
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


这里我们测试3折交叉验证过程的平均RMSE和MAE，但也可以使用任何交叉验证迭代器。

一旦fit()被调用，best_estimator会使我们得到参数被优化过的算法实例，我们可以这样使用：

In [16]:
algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

注意：字典参数，例如bsl_options与sim_options需特殊对待。请参考以下例子：

In [17]:
param_grid = {'k': [10, 20],
              'sim_options': {'name': ['msd', 'cosine'],
                              'min_support': [1, 5],
                              'user_based': [False]}
              }

当然，两者可以结合，例如 KNNBaseline ：

In [18]:
param_grid = {'bsl_options': {'method': ['als', 'sgd'],
                              'reg': [1, 2]},
              'k': [2, 3],
              'sim_options': {'name': ['msd', 'cosine'],
                              'min_support': [1, 5],
                              'user_based': [False]}
              }

进一步分析，cv_results包括所有的信息而且可以被导入到一个pandas的dataframe。

In [21]:
gs.cv_results
# 你可以发现，每个列表具有相同大小的参数组合数，它对应下面的表格（参见官方文档）

{'split0_test_rmse': array([1.00083517, 1.00644857, 0.97675819, 0.98583461, 0.98118718,
        0.98940678, 0.96724891, 0.9770976 ]),
 'split1_test_rmse': array([0.99798113, 1.0039441 , 0.97524208, 0.98394272, 0.97934397,
        0.98735938, 0.96604354, 0.97574438]),
 'split2_test_rmse': array([0.99439787, 1.00052021, 0.97056491, 0.97928479, 0.9749666 ,
        0.98307795, 0.96030902, 0.97005727]),
 'mean_test_rmse': array([0.99773806, 1.00363763, 0.97418839, 0.9830207 , 0.97849925,
        0.98661471, 0.96453382, 0.97429975]),
 'std_test_rmse': array([0.00263363, 0.00242992, 0.00263589, 0.00275229, 0.00260884,
        0.00263685, 0.00302764, 0.00305033]),
 'rank_test_rmse': array([7, 8, 2, 5, 4, 6, 1, 3]),
 'split0_test_mae': array([0.80866005, 0.81709885, 0.78444321, 0.79563627, 0.78851038,
        0.79916104, 0.77543641, 0.78732908]),
 'split1_test_mae': array([0.80575347, 0.81434446, 0.78190872, 0.79279032, 0.78587982,
        0.79634603, 0.77284442, 0.78458049]),
 'split2_test_mae

In [22]:
results_df = pd.DataFrame.from_dict(gs.cv_results)
results_df

,split0_test_rmse,split1_test_rmse,split2_test_rmse,mean_test_rmse,std_test_rmse,rank_test_rmse,split0_test_mae,split1_test_mae,split2_test_mae,mean_test_mae,std_test_mae,rank_test_mae,mean_fit_time,std_fit_time,mean_test_time,std_test_time,params,param_n_epochs,param_lr_all,param_reg_all
0,1.000835,0.997981,0.994398,0.997738,0.002634,7,0.808660,0.805753,0.804968,0.806461,0.001588,7,1.039868,0.040576,0.274348,0.051151,"{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.4}",5,0.002,0.4
1,1.006449,1.003944,1.000520,1.003638,0.002430,8,0.817099,0.814344,0.813388,0.814944,0.001573,8,1.004233,0.022924,0.303481,0.055552,"{'n_epochs': 5, 'lr_all': 0.002, 'reg_all': 0.6}",5,0.002,0.6
2,0.976758,0.975242,0.970565,0.974188,0.002636,2,0.784443,0.781909,0.780576,0.782309,0.001604,2,1.020309,0.031337,0.261619,0.056405,"{'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.4}",5,0.005,0.4
3,0.985835,0.983943,0.979285,0.983021,0.002752,5,0.795636,0.792790,0.791563,0.793330,0.001706,5,1.007655,0.031823,0.262539,0.054900,"{'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.6}",5,0.005,0.6
4,0.981187,0.979344,0.974967,0.978499,0.002609,4,0.788510,0.785880,0.784872,0.786421,0.001534,4,2.032386,0.067592,0.277278,0.051892,"{'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.4}",10,0.002,0.4
5,0.989407,0.987359,0.983078,0.986615,0.002637,6,0.799161,0.796346,0.795199,0.796902,0.001665,6,1.960554,0.038695,0.283283,0.043981,"{'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.6}",10,0.002,0.6
6,0.967249,0.966044,0.960309,0.964534,0.003028,1,0.775436,0.772844,0.770931,0.773071,0.001846,1,2.061431,0.061431,0.259438,0.049942,"{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}",10,0.005,0.4
7,0.977098,0.975744,0.970057,0.974300,0.003050,3,0.787329,0.784580,0.782740,0.784883,0.001886,3,1.973532,0.054076,0.288961,0.048240,"{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.6}",10,0.005,0.6


# 预测算法的使用

Surprise提供很多内置算法。所有算法都来源于AlgoBase这个基础类，这其中包括一些关键方法（例如predict，fit和test）可供使用的预测算法的种类与细节可以在prediction_algorithms文档中查阅。

每个算法都是Surprise全局命名空间的一部分，因此您只需要从Surprise包中导入它们的名称，例如：

```python
from surprise import KNNBasic
algo = KNNBasic()
```

一些算法可能使用baseline estimates，一些算法可能使用similarity measure。我们在这里回顾 baseline与 similarities的计算方式。

# Baselines estimates configuration

注意：

本节仅适用于那些最小化以下正则化平方误差的算法（或similarity measures）：

对于其他使用baseline目标函数的算法（例如SVD算法），配置是完全不同的，请查阅它们自己的文档进行了解。

首先，如果你不想了解baseline是如何计算的，默认的参数完全可以供您直接使用。如果您想深入了解，那请继续阅读。

我们可以通过两种不同的方式计算baseline：

* 1. Stochastic Gradient Descent (SGD)。
* 2. Alternating Least Squares (ALS)。

您可以使用在创建算法时使用参数bsl_options来设置baseline的计算方式，此参数由一个字典表示，其键method确定要使用的方法，可允许接受的值为als（默认）和sgd。根据你所选择的值，可以进一步设置其他参数。

对于ALS:
* reg_i：物品的正则化参数，对应Kor10中的参数λ2 ，默认值为10。
* reg_u：用户的正则化参数，对应Kor10中的参数λ3，默认值为15 。
* n_epochs：ALS程序迭代的次数，默认值为10，注意Kor10所描述的是单次ALS的过程。

对于SGD：
* reg：代价函数的正则化项，对应Kor10中的λ1与 λ5，默认值为0.02。
* learning_rate：SGD的学习率，对应Kor10中的γ ，默认值0.005。
* n_epochs：SGD的迭代次数，默认值为20。

注意：对于ALD与SGD,用户和物品的偏置（bu和bi）初始化为都为0。

ALS:
```python
print('Using ALS')
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
algo = BaselineOnly(bsl_options=bsl_options)
```

SGD:
```python
print('Using SGD')
bsl_options = {'method': 'sgd',
               'learning_rate': .00005,
               }
algo = BaselineOnly(bsl_options=bsl_options)
```

# Similarity measure configuration

许多算法使用相似度来估计评分的算法，配置方法和baseline rating类似，在调用算法时，只需传递sim_options参数进行配置，这个参数的形式必须是字典，介绍关键的参数如下：

* name：相似度度量方法名称， 这些方法定义在similarities中，默认是MSD。
* user_based: 用户或者物品间相似度的度量。默认为True（基于用户）。
* min_support:（没搞懂，待补充）
* shrinkage：此参数仅在使用pearson_baseline相似度时设置，默认100。

举例：
```python
sim_options = {'name': 'cosine',
               'user_based': False  # compute  similarities between items
               }
algo = KNNBasic(sim_options=sim_options)
```

```python
sim_options = {'name': 'pearson_baseline',
               'shrinkage': 0  # no shrinkage
               }
algo = KNNBasic(sim_options=sim_options)
```
详情请参考similarities模型。

# 案例分析

## 使用surprise框架为Movieslen数据集中的每个user推荐Top-N个item

surprise是Simple Python Recommendation System Engine的缩写，是一个为了实现推荐系统的框架。它自带了SVD，user-based，item-based协同过滤算法等多种推荐算法，接口简单，功能强大。但官方文档写的并不好，笔者花了不少时间，都没有找到如何预测某个user对某个item进行打分这样的基础用法，所以把摸索后得到的经验分享于此。

某个user对某个item的评分
如何实现user-5对item-i打分的预测值?

首先，导入surprise中必须的class：SVD是SVD算法，Dataset是创建满足surprise需要的数据集所需的类，Reader是做数据读取，类似pandas.

这里要注意的是，Dataset结构并不是surprise结构直接进行计算的最终结构，还需要通过data.build_full_trainset()将其转换为Trainset这样的数据结构，才能在后续过程中直接用于训练。

这里用SVD对数据进行训练，也就是SVD推荐算法，将原始得分矩阵分解后，对未知得分进行重新计算的过程。

接下来，我们就能计算user-5对item-1的打分（预测值）了。这里要注意，user-id和item-id，都要将id转换为string类型。然后调用algo.predict(uid, iid)就能得到预测值了。

可以迭代用algo.predict(uid, iid)对每个得分进行计算，然后遍历所有得分得到TOP-N。

但这样做稍显笨拙，其实surprise作为一个优秀的推荐系统框架，已经给出了更好的接口，下文进行详述。

* 首先是读入数据，转换为Trainset结构，并用SVD算法进行训练。这个过程和上一节的内容相同。

* 接下来定义get_top_n()函数，它能根据predictions结果进行解析，获取top_n字典，该字典的key是user-id，value是该user打分（预测值）最高的n个item-id。predictions的数据结构，是surprise中的算法自带接口algo.test()的输出值。

In [33]:
#导入相关的库文件
import os
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import train_test_split

In [26]:
##读取数据
#指定文件的路径
file_path = os.path.expanduser('ml-100k/u.data')
#告诉文本阅读器，文本的格式是什么样子的
reader = Reader(line_format='user item rating timestamp', sep='\t')
#加载数据
data = Dataset.load_from_file(file_path, reader=reader)

# testset占比25%.
#trainset, testset = train_test_split(data, test_size=.25)
trainset = data.build_full_trainset()
#这里使用SVD算法,也可以使用其他的算法
algo = SVD()
# 在trainset上进行模型的训练, 在testset进行预测
algo.fit(trainset)
#进行预测
testset = trainset.build_anti_testset()  #这里的testset是trainset中 rui为0的（user, item， 0）
predictions = algo.test(testset)
# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.6053


0.6053469102940916

In [28]:
predictions[:3]

[Prediction(uid='196', iid='302', r_ui=3.52986, est=4.185470162932979, details={'was_impossible': False}),
 Prediction(uid='196', iid='377', r_ui=3.52986, est=2.6010096338421476, details={'was_impossible': False}),
 Prediction(uid='196', iid='51', r_ui=3.52986, est=3.5050179070196315, details={'was_impossible': False})]

In [29]:
from collections import defaultdict       #defaultdict是一个字典，当key不存在时，会返回默认值
#从一个prediction集合中返回每个 user Top-N推荐
def get_top_n(predictions, n = 10):
    '''从一个prediction集合中返回每个 user Top-N推荐
    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.
    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''
    #首先将prediction映射到每个user上
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    #再对每个user的item按照评分进行排序
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key = lambda x : x[1], reverse = True)
        top_n[uid] = user_ratings[:n]   #取前n个
    return top_n

get_top_n(predictions)

defaultdict(list,
            {'196': [('169', 4.607089703642116),
              ('114', 4.591126206457514),
              ('12', 4.503855829408288),
              ('603', 4.435840868909902),
              ('657', 4.384955734419694),
              ('654', 4.370433255334769),
              ('187', 4.36542431978906),
              ('483', 4.355683709519551),
              ('194', 4.35476543861892),
              ('50', 4.34381270916753)],
             '186': [('659', 4.684552730031168),
              ('316', 4.554849878484988),
              ('114', 4.536877600736244),
              ('318', 4.518042735758555),
              ('489', 4.473477086294479),
              ('480', 4.470395475106738),
              ('513', 4.468883737641556),
              ('408', 4.461695677737885),
              ('498', 4.445261511983604),
              ('143', 4.430816928352844)],
             '22': [('135', 4.896136618179206),
              ('12', 4.817077662751386),
              ('483', 4.795476956739311),


## 计算查准率 precision@k 和查全率 recall@k

准确率，顾名思义，就是准确程度。通过正确数/总数得到。而正确数是什么，总数是什么呢？

召回率，我们可以理解为找到的数目与总的需要我们找到的数目的比，那在推荐系统中，什么是找到的数目，什么是需要我们总的找到的数目呢？

**令R(u)表示在根据训练数据给用户做出的推荐列表，T(u)表示用户根据测试数据给用户做出的推荐列表**，则

召回率：

![img](https://img-blog.csdn.net/20170306203003766)

准确率：

![img](https://img-blog.csdn.net/20170306203017351)

**精确率是针对我们预测结果而言的，它表示的是预测为正的样本中有多少是真正的正样本**，例如我们给用户推荐了100条新闻，其中10条用户产生了点击，那么准确率为10/100 = 0.1

而**召回率是针对我们原来的样本而言的，它表示的是样本中的正例有多少被预测正确了**， 例如我们给用户推荐了100条新闻，其中10条用户产生了点击，而用户最终在平台上总共点击了200条新闻，那么召回率为10 / 200 = 0.05， **表示的是推荐系统推荐的那些符合用户兴趣并产生点击的新闻量占了用户实际总共点击的新闻 有多少比例** 

![img](https://img-blog.csdn.net/20170306204344111)

其实就是分母不同，**一个分母是预测为正的样本数，另一个是原来样本中所有的正样本数**。但**分子都是表示预测的正样本与原来正样本的交集。**

![img](https://img-blog.csdn.net/20170306213045080?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvdTAxMzU5OTI5OA==/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/Center)

在信息检索领域，精确率和召回率又被称为**查准率和查全率**，

* 查准率＝检索出的相关信息量 / 检索出的信息总量
* 查全率＝检索出的相关信息量 / 系统中的相关信息总量